## Extracción de caracteristicas usando SIFT

In [1]:
import matplotlib.pyplot as plt
from urllib.request import urlopen
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
import pandas as pd
import json
import cv2
import os

def extract_sift_features(image_url):
    """
    Extrae características SIFT de una imagen desde URL
    """
    try:
        img = imread(image_url) # Obtener imagen desde URL
        if len(img.shape) == 3:
            img = rgb2gray(img)

        img = (img * 255).astype(np.uint8)
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(img, None)

        if descriptors is None:
            return None, None

        mean_descriptor = np.mean(descriptors, axis=0)
        return keypoints, mean_descriptor

    except Exception as e:
        print(f"Error procesando imagen: {e}")
        return None, None


def process_and_save_incremental(csv_file, output_dir):
    """
    Procesa y guarda características de forma incremental, reanudando desde el último checkpoint
    """
    os.makedirs(output_dir, exist_ok=True)
    checkpoint_file = f"{output_dir}/checkpoint.json"
    descriptors_file = f"{output_dir}/descriptores.npy"

    # Cargar estado anterior si existe
    processed = {}
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            processed = json.load(f)

    descriptors_list = []
    if os.path.exists(descriptors_file):
        descriptors_list = list(np.load(descriptors_file))

    df = pd.read_csv(csv_file)
    failed_images = []

    print(f"Iniciando procesamiento desde imagen {len(processed)}")

    for idx, (filename, image_url) in enumerate(zip(df['filename'], df['link'])):
        if filename in processed:
            continue

        print(f"Procesando {idx+1}/{len(df)}: {filename}")

        try:
            keypoints, descriptor = extract_sift_features(image_url)

            if descriptor is not None:
                descriptors_list.append(descriptor)
                processed[filename] = len(descriptors_list) - 1

                np.save(descriptors_file, np.vstack(descriptors_list))
                with open(checkpoint_file, 'w') as f:
                    json.dump(processed, f)

                print(f"Guardado - Total procesadas: {len(processed)}")
            else:
                failed_images.append(filename)
                print(f"Fallo al procesar: {filename}")

        except Exception as e:
            print(f"Error en {filename}: {e}")
            failed_images.append(filename)

        # Guardar lista de fallos
        with open(f"{output_dir}/failed.json", 'w') as f:
            json.dump(failed_images, f)

    print("\nProcesamiento completado:")
    print(f"- Total procesadas: {len(processed)}")
    print(f"- Fallos: {len(failed_images)}")

    return np.vstack(descriptors_list), processed, failed_images




In [2]:

# def load_features(output_dir):
#     """
#     Carga descriptores y mapeo desde archivos
#     """
#     descriptors = np.load(f"{output_dir}/descriptores.npy")
#     with open(f"{output_dir}/checkpoint.json", 'r') as f:
#         mapping = json.load(f)
#     return descriptors, mapping

In [ ]:
# Uso
output_dir = "features"
descriptors, mapping, failed = process_and_save_incremental("images.csv", output_dir)

# Cargar cuando sea necesario
# descriptors, mapping = load_features(output_dir)

Iniciando procesamiento desde imagen 3403
Procesando 3404/44446: 41267.jpg
Guardado - Total procesadas: 3404
Procesando 3405/44446: 11517.jpg
Guardado - Total procesadas: 3405
Procesando 3406/44446: 25722.jpg
Guardado - Total procesadas: 3406
Procesando 3407/44446: 55455.jpg
Guardado - Total procesadas: 3407
Procesando 3408/44446: 12392.jpg
Guardado - Total procesadas: 3408
Procesando 3409/44446: 31110.jpg
Guardado - Total procesadas: 3409
Procesando 3410/44446: 2617.jpg
Guardado - Total procesadas: 3410
Procesando 3411/44446: 43530.jpg
Guardado - Total procesadas: 3411
Procesando 3412/44446: 13240.jpg
Guardado - Total procesadas: 3412
Procesando 3413/44446: 54587.jpg
Guardado - Total procesadas: 3413
Procesando 3414/44446: 27075.jpg
Guardado - Total procesadas: 3414
Procesando 3415/44446: 46042.jpg
Guardado - Total procesadas: 3415
Procesando 3416/44446: 7365.jpg
Guardado - Total procesadas: 3416
Procesando 3417/44446: 49836.jpg
Guardado - Total procesadas: 3417
Procesando 3418/44446: